<font size="6"><b>S18 T01: Tasca base de dades NoSQL</b></font>


# Enunciats
**Nivell 1**

**Exercici 1**

Crea una base de dades NoSQL utilitzant MongoDB. Afegeix-li algunes dades d'exemple que et permetin comprovar que ets capaç de processar-ne la informació de manera bàsica.

**Exercici 2**

Connecta la base de dades NoSQL a Python utilitzant per exemple pymongo.

**Nivell 2**  

**Exercici 1**

Carega algunes consultes senzilles a un Pandas Dataframe. 


**Nivell 3**  

**Exercici 1**  


Genera un resum estadístic de la informació que conté la base de dades.

# Exercicis

Desde el Jupyter Notebook i fent servir Python i les seves llibreis (Pymongo) interactuaré amb MongoDB, creant arxius i pujant una col.lecció.

Després, guardarem les dades fruit d'una consulta a un Pandas Dataframe, per poder fer l'anàlisi estadístic.

In [2]:
import pymongo
import pandas as pd 
from pandas import json_normalize

In [1]:

#conecctem amb el meu client de MongoDB

client = pymongo.MongoClient('mongodb://localhost:27017/')

In [3]:
# preparem el fitxer q farem servir com a colecction

df = pd.read_csv('sell_through.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [4]:
#es converteix a JSON
data = df.to_dict(orient='records')

In [6]:
#es crea la base de dades i l'omplim amb les ades del diccionari

db = client['TascaDivuitU']
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'TascaDivuitU')


In [7]:
db.Vendes.insert_many(data)

In [9]:
db.Vendes.count_documents({})

5681

In [10]:
db.Vendes.find_one()

{'_id': ObjectId('6145b11d6e3bbbf816528b35'),
 'Item_Identifier': 'FDW58',
 'Item_Weight': 20.75,
 'Item_Fat_Content': 'Low Fat',
 'Item_Visibility': 0.007564836,
 'Item_Type': 'Snack Foods',
 'Item_MRP': 107.8622,
 'Outlet_Identifier': 'OUT049',
 'Outlet_Establishment_Year': 1999,
 'Outlet_Size': 'Medium',
 'Outlet_Location_Type': 'Tier 1',
 'Outlet_Type': 'Supermarket Type1'}

In [17]:
# seleccionarem alguns camps dels articles amb contigut de greix Regular.

cerca = [i for i in db.Vendes.find({'Item_Fat_Content':{'$regex':'^reg'}}, 
                                     {'_id':0,'Item_Identifier': 1,'Item_Weight':1,'Item_Visibility': 1,'Item_Type': 1, 'Item_MRP': 1 ,'Outlet_Identifier': 1})]

In [18]:
# guardem les dades obtingudes a un df
df = json_normalize(cerca)

In [19]:
df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier
0,FDW14,8.300,0.038428,Dairy,87.3198,OUT017
1,FDG12,6.635,0.006326,Baking Goods,122.1098,OUT046
2,FDJ10,5.095,0.216763,Snack Foods,142.2838,OUT010
3,FDM27,12.350,0.000000,Meat,158.8946,OUT046
4,FDK32,16.250,0.048935,Fruits and Vegetables,153.0682,OUT013


In [20]:
#ara ja només ens queda aplicar el mètode describe per obtenir les dades estadíastciques d'aquesta nova taula.
df.describe()

,Item_Weight,Item_Visibility,Item_MRP
count,65.000000,78.000000,78.000000
mean,12.230385,0.071744,136.062438
std,4.697225,0.060479,64.026940
min,5.095000,0.000000,34.553200
25%,8.300000,0.026789,87.369150
50%,11.395000,0.054084,126.052000
75%,16.100000,0.103286,185.366100
max,20.600000,0.308259,260.593600


In [21]:
# seleccionarem alguns camps dels articles amb contigut de greix Regular i siguin del tipus llet o derivats.

llet = [i for i in db.Vendes.find({'Item_Fat_Content':{'$regex':'^reg'},'Item_Type':'Dairy' }, 
                                     {'_id':0,'Item_Identifier': 1,'Item_Weight':1,'Item_Visibility': 1,'Item_Type': 1, 'Item_MRP': 1 ,'Outlet_Identifier': 1})]

In [22]:
df = json_normalize(llet)

In [23]:
df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier
0,FDW14,8.300,0.038428,Dairy,87.3198,OUT017
1,FDA02,14.000,0.029891,Dairy,146.3786,OUT017
2,FDC03,8.575,0.071846,Dairy,196.1794,OUT046
3,FDL51,NaN,0.047261,Dairy,213.5876,OUT027
4,FDY38,13.600,0.119154,Dairy,231.2300,OUT035


In [24]:
#ara ja només ens queda aplicar el mètode describe per obtenir les dades estadíastciques d'aquesta nova taula.
df.describe()

,Item_Weight,Item_Visibility,Item_MRP
count,7.000000,8.000000,8.000000
mean,11.396429,0.059527,163.474950
std,4.808769,0.036716,67.022987
min,6.750000,0.022493,87.319800
25%,8.300000,0.036189,94.761350
50%,8.575000,0.042845,171.279000
75%,13.800000,0.081098,217.998200
max,20.250000,0.119154,247.809200
